In [1]:
import numpy as np
import math
from scipy.stats import unitary_group
from skimage.measure import block_reduce
from opt_einsum import contract
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import torch
import time
import sys
import matplotlib.pyplot as plt
from matplotlib import rc
from src.QDDPM_torch import DiffusionModel, QDDPM, naturalDistance
#import src.ImageEncode as ie
rc('text', usetex=False)
rc('axes', linewidth=3)

Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [10]:
def angle_encode(image: np.array):
    Npixels = np.size(image)

    image = image.flatten()

    #if ((Npixels & (Npixels - 1)) == 0 and Npixels != 0):
    #    print("Pixel number is not a power of 2")
    #    return -1

    states = (1 + 1j) * np.zeros(2 ** Npixels)
    scale_val = (0.5 * math.pi / np.max(image))
    image = image * scale_val

    for i in range(0, 2 ** Npixels):
        states[i] = 1 + 0j
        temp = i
        for j in range(0, Npixels):
            if (temp % 2):
                states[i] *= math.sin(image[Npixels - 1 - j])
            else:
                states[i] *= math.cos(image[Npixels - 1 - j])
            temp = int(temp / 2)
    
    return scale_val, states

In [15]:
#test angle_encode
temp_test = np.array([1, 1])
#plt.imshow(temp_test, cmap='grey',interpolation = 'nearest')
scale_val, states = angle_encode(temp_test)
print(states)

[1.00000000e+00+0.j 6.12323400e-17+0.j 6.12323400e-17+0.j
 3.74939946e-33+0.j]


In [1]:
def angle_decode(states: np.array, scale_val: float):
    Npixels = int(math.log2(np.size(states)))
    image_dim = int(Npixels ** 0.5)
    num_states = np.size(states)

    if(image_dim != (Npixels ** 0.5)):
        print("image is not n by n pixels")
        return -1

    image_flattened = np.array([])
    image_nxn = np.zeros((image_dim, image_dim))

    #get rid of complex parts and square the array to get sin^2 and cos^2 components
    states = np.square(np.abs(states))

    #create intermediate array to do calculations
    temp_copy = states

    while(np.size(image_flattened) != Npixels):
        
        if(np.size(temp_copy) == 2):
            #first elements of temp_copy will always be cos^2(x * scale_val), append x to image_flattened
            image_flattened = np.append(image_flattened, math.acos(temp_copy[0] ** 0.5)) / scale_val

            #variable for how many indices it takes to switch divisor between cos and sin
            len_switch = num_states / (2 ** np.size(image_flattened)) #as we find each MSQB value the length between switches will half
            count = 0
            divisor_cos = temp_copy[0]
            divisor_sin = temp_copy[1]
            temp_copy = states[0:np.size(states)/2] #only copy half the remaining states
            if(np.size(temp_copy) >= 2):
                for k in range(0, np.size(temp_copy)):
                    if(count < len_switch):
                        temp_copy[k] /= divisor_cos
                        count += 1
                    else:
                        temp_copy[k] /= divisor_sin
                        count += 1
                    if(count == len_switch * 2):
                        count = 0

                states = temp_copy

        while(np.size(temp_copy) > 2):
            #add neighboring array values to cancel out LSQB (sin^2 + cos^2 = 1)
            for j in range(0, np.size(temp_copy)):
                temp_copy[j] += temp_copy[j+1]
                np.delete(temp_copy, j+1)

    for i in range(0, image_dim):
        for j in range(0, image_dim):
            image_nxn[i][j] = image_flattened[i * image_dim + j]

    return image_nxn

KeyboardInterrupt: 